In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'

Mounted at /content/gdrive


In [ ]:
!pip install torchvision torchaudio rouge-score scipy numpy


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=875d6381e018b0c1b84b734fe9415d0a99018dcdcbd80f1075db6d6537568b97
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
!python gdrive/My\ Drive/train.py \
    --train_file "gdrive/My Drive/train.json" \
    --valid_file "gdrive/My Drive/valid.json" \
    --batch_size_train 8 \
    --batch_size_valid 4 \
    --model_file "gdrive/My Drive/t5/model" \
    --learning_rate 1e-5 \
    --warmup_steps 16000 \
    --num_epochs 3 \
    --ckpt_dir "gdrive/My Drive/checkpoints/t5" \
    --ckpt_name "t5Base" \
    --device "cpu"

2025-01-05 14:50:01.882447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-05 14:50:01.923048: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-05 14:50:01.934959: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 14:50:01.965772: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-05 14:50:03.940468: W tensorflow/compiler/tf2

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import json
from transformers import BartTokenizer, BartForConditionalGeneration,AutoModelForSeq2SeqLM, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration

In [8]:
import json

# Load the JSON data
def load_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data

# Example usage
data = load_data("gdrive/My Drive/train.json")


In [9]:
# Summarize the dataset
def summarize_data(data):
    num_records = len(data)
    example_questions = [item["question"] for item in data[:3]]
    labels = [key for key in data[0].get("labelled_answer_spans", {}).keys()]

    summary = {
        "Total Records": num_records,
        "Example Questions": example_questions,
        "Labels": labels
    }
    return summary

# Display summary
summary = summarize_data(data)
print(summary)


{'Total Records': 2236, 'Example Questions': ['what is parkinesonism?', 'I scream, shout and swear in my sleep. How do I stop?', 'what does m.c.h.c stand for in my hematology test? I wont be able to see my dr. for another week and it bother'], 'Labels': ['INFORMATION']}


In [10]:
# Preprocess the data
def preprocess_data(data):
    processed_data = []
    for item in data:
        processed_data.append({
            "uri": item["uri"],
            "question": item["question"],
            "context": item["context"],
            "answers": item["answers"],
            "labelled_spans": item.get("labelled_answer_spans", {}),
            "summaries": item.get("labelled_summaries", {})
        })
    return processed_data

# Example usage
processed_data = preprocess_data(data)


In [11]:
from transformers import pipeline

# Initialize summarization model
def init_model(model_name="t5-small"):
    summarizer = pipeline("summarization", model=model_name)
    return summarizer

# Example usage
summarizer = init_model()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


In [14]:
# Input raw_text and labelled text
raw_text = """uri: 744844
question: Why do they tell you not to take vitamens before a blood test?
context: Im getting a check-up for life insurance and they said no vitamens just curious.
answer_0: I agree. Some diseases are a result of vitamin deficiencies so if you take vitamins that are normally lacking you could end up with a misdiagnosis.
answer_1: Depends on the doses. It can mask symptoms of deficiency as well as give false-positives on other tests. Seriously, don't do it.
INFORMATION_GROUP: Some diseases are a result of vitamin deficiencies so if you take vitamins that are normally lacking you could end up with a misdiagnosis.: Depends on the doses. It can mask symptoms of deficiency as well as give false-positives on other test

SUGGESTION_GROUP: Seriously, don't do it
"""
labelled_text = "Depends on the doses. It can mask symptoms of deficiency as well as give false-positives on other test"

# Compute span
start_index = raw_text.find(labelled_text)
end_index = start_index + len(labelled_text)

# Output span
label_span = [start_index, end_index]
print("Label span:", label_span)


Label span: [343, 445]


In [21]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")  # Ensure the NLTK sentence tokenizer is available

# Input data
data = {
    "answers": [
        "I agree. Some diseases are a result of vitamin deficiencies so if you take vitamins that are normally lacking you could end up with a misdiagnosis.",
        "Depends on the doses. It can mask symptoms of deficiency as well as give false-positives on other tests. Seriously, don't do it."
    ],
    "labelled_answer_spans": {
        "INFORMATION": [
            {
                "txt": "Some diseases are a result of vitamin deficiencies so if you take vitamins that are normally lacking you could end up with a misdiagnosis",
                "label_spans": [194, 331]
            },
            {
                "txt": "Depends on the doses. It can mask symptoms of deficiency as well as give false-positives on other test",
                "label_spans": [341, 445]
            }
        ],
        "SUGGESTION": [
            {"txt": "Seriously, don't do it", "label_spans": [448, 470]}
        ]
    }
}

# Map for perspective labels
perspective_map = {
    'CAUSE': 1,
    'SUGGESTION': 2,
    'EXPERIENCE': 3,
    'QUESTION': 4,
    'INFORMATION': 5,
    'NONE': 0
}

# Preprocess labeled spans: tokenize and map
labeled_sentences = {}
for category, spans in data["labelled_answer_spans"].items():
    label = perspective_map[category]
    for span in spans:
        sentences = sent_tokenize(span["txt"].strip())  # Split into sentences
        for sentence in sentences:
            labeled_sentences[sentence.strip()] = label

# Process answers and split into sentences
processed_output = {}
for answer in data["answers"]:
    sentences = sent_tokenize(answer)  # Split into sentences
    for sentence in sentences:
        sentence = sentence.strip()

        # Check if the sentence matches any labeled spans
        matched_label = 0  # Default label is NONE
        for span_text, label in labeled_sentences.items():
            if span_text in sentence or sentence in span_text:  # Match or overlap check
                matched_label = label
                break

        processed_output[sentence] = matched_label

# Display the output
print(processed_output)


{'I agree.': 0, 'Some diseases are a result of vitamin deficiencies so if you take vitamins that are normally lacking you could end up with a misdiagnosis.': 5, 'Depends on the doses.': 5, 'It can mask symptoms of deficiency as well as give false-positives on other tests.': 5, "Seriously, don't do it.": 2}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
import nltk
from nltk.tokenize import sent_tokenize
import json
import pandas as pd

# Ensure the NLTK sentence tokenizer is available
nltk.download("punkt")

# Function to process the JSON data
def process_json_data(data):
    """
    Process the input JSON data, split answers into sentences and apply corresponding tags.

    Args:
        data: The JSON data containing 'answers' and 'labelled_answer_spans'

    Returns:
        processed_output: Dictionary of sentences and their corresponding labels
    """
    # Map for perspective labels
    perspective_map = {
        'CAUSE': 1,
        'SUGGESTION': 2,
        'EXPERIENCE': 3,
        'QUESTION': 4,
        'INFORMATION': 5,
        'NONE': 0
    }

    # Preprocess labeled spans: tokenize and map
    labeled_sentences = {}
    for category, spans in data["labelled_answer_spans"].items():
        label = perspective_map[category]
        for span in spans:
            sentences = sent_tokenize(span["txt"].strip())  # Split into sentences
            for sentence in sentences:
                labeled_sentences[sentence.strip()] = label

    # Process answers and split into sentences
    processed_output = {}
    for answer in data["answers"]:
        sentences = sent_tokenize(answer)  # Split into sentences
        for sentence in sentences:
            sentence = sentence.strip()

            # Check if the sentence matches any labeled spans
            matched_label = 0  # Default label is NONE
            for span_text, label in labeled_sentences.items():
                if span_text in sentence or sentence in span_text:  # Match or overlap check
                    matched_label = label
                    break

            processed_output[sentence] = matched_label

    return processed_output


# Function to process the entire JSON file and save it
def process_and_save_data(json_file, output_csv):
    # Read the JSON file
    with open(json_file, "r") as file:
        raw_data = json.load(file)

    # List to store the results for all entries
    all_results = []

    # Process each entry in the JSON file
    for entry in raw_data:
        processed_result = process_json_data(entry)

        # Convert the result to a format suitable for saving
        for sentence, label in processed_result.items():
            all_results.append({"sentence": sentence, "label": label})

    # Convert the results into a DataFrame and save it as CSV
    df = pd.DataFrame(all_results)
    df.to_csv(output_csv, index=False)
    print(f"Cleaned data saved to {output_csv}")


# Example Usage
json_file = "gdrive/My Drive/train.json"  # Replace with your JSON file path
output_csv = "gdrive/My Drive/cleaned_data.csv"  # Output CSV file name

# Process the data and save to CSV
process_and_save_data(json_file, output_csv)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Cleaned data saved to gdrive/My Drive/cleaned_data.csv


In [23]:
!pip install transformers scikit-learn torch


In [28]:
!pip install sentence-transformers

In [24]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [25]:
# Load your dataset
data = pd.read_csv("gdrive/My Drive/cleaned_data.csv")  # Replace with your dataset file
sentences = data['sentence'].tolist()
labels = data['label'].tolist()

In [26]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)


In [27]:
# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    sentences, encoded_labels, test_size=0.2, random_state=42
)

In [30]:
from sentence_transformers import SentenceTransformer

# Load the all-MiniLM-L6-v2 model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
# Generate sentence embeddings
embeddings = embedding_model.encode(sentences, convert_to_numpy=True)

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

In [32]:
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

In [33]:
# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

In [34]:
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


In [35]:
# Define neural network
class SentenceClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SentenceClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [36]:
# Initialize model
input_dim = X_train.shape[1]
num_classes = len(set(labels))  # Number of unique labels (e.g., 6)
model = SentenceClassifier(input_dim=input_dim, num_classes=num_classes)

In [37]:
# Training configuration
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [38]:
# Train the model
for epoch in range(5):  # Number of epochs
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs, targets = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 1452.7382
Epoch 2, Loss: 1303.5971
Epoch 3, Loss: 1261.9426
Epoch 4, Loss: 1225.3335
Epoch 5, Loss: 1192.3161


In [39]:
# Evaluate on validation set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in val_loader:
        inputs, targets = batch
        outputs = model(inputs)
        _, predicted = torch.max(outputs, dim=1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

print(f"Validation Accuracy: {correct / total:.4f}")

Validation Accuracy: 0.6205


In [40]:
# Save the trained model
torch.save(model.state_dict(), "gdrive/My Drive/sentence_classifier.pt")


In [41]:
# Load the model architecture
input_dim = 384  # For all-MiniLM-L6-v2, the embedding size is 384
num_classes = 6  # Replace with the number of your classes
model = SentenceClassifier(input_dim=input_dim, num_classes=num_classes)

# Load the saved weights
model.load_state_dict(torch.load("gdrive/My Drive/sentence_classifier.pt"))
model.eval()  # Set the model to evaluation mode


<ipython-input-41-e5e896ff6120>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("gdrive/My Drive/sentence_classifier.pt"))


SentenceClassifier(
  (fc1): Linear(in_features=384, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=6, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
)

In [42]:
correct = 0
total = 0
with torch.no_grad():
    for batch in val_loader:
        inputs, targets = batch
        outputs = model(inputs)
        _, predicted = torch.max(outputs, dim=1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

print(f"Validation Accuracy: {correct / total:.4f}")

Validation Accuracy: 0.6205


In [43]:
!pip install joblib


In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Train Logistic Regression model
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Predict on validation set
y_pred = clf.predict(X_val)

# Evaluate the model
print(classification_report(y_val, y_pred))


              precision    recall  f1-score   support

           0       0.57      0.40      0.47      1047
           1       0.63      0.23      0.34       113
           2       0.61      0.62      0.62      1341
           3       0.62      0.44      0.51       575
           4       0.82      0.25      0.38        73
           5       0.61      0.80      0.69      1937

    accuracy                           0.61      5086
   macro avg       0.65      0.45      0.50      5086
weighted avg       0.61      0.61      0.59      5086

